# Metadata summary table

In [1]:
import os
import sys
from socket import gethostname

# this is for ffmpeg
os.environ['PATH'] += os.pathsep + os.path.expanduser('~/miniconda3/envs/bioinformatics/bin')

hostname = gethostname()

if hostname.startswith('hpc-node'):
    IN_DEEPTHOUGHT = True
    sys.path.append('..')
else:
    IN_DEEPTHOUGHT = False
from cf_analysis_lib.load_libraries import *
import cf_analysis_lib


In [2]:
sequence_type = "MGI"
datadir = '..'
sslevel = 'subsystems_norm_ss.tsv.gz'
taxa = "family"

df, metadata = cf_analysis_lib.read_the_data(sequence_type, datadir, sslevel='subsystems_norm_ss.tsv.gz', taxa="family")

Using ../MGI/FunctionalAnalysis/subsystems/MGI_subsystems_norm_ss.tsv.gz for the subsystems


## Person-level summaries

In [3]:
meta = metadata.copy()
# Ensure dates are datetime
meta['Sample date'] = pd.to_datetime(meta['Sample date'])
# One row per person for person-level summaries
persons = meta.sort_values('Sample date').groupby('pwCF_ID', as_index=False).first()

## Cohort size, samples, study period

In [4]:
n_people = meta['pwCF_ID'].nunique()
n_samples = len(meta)

start_date = meta['Sample date'].min()
end_date = meta['Sample date'].max()
duration_days = (end_date - start_date).days  # 528 in your text
duration_days

528

## Sex distribution (per person)

In [5]:
sex_counts = persons['Gender'].value_counts().sort_index()
# e.g. Female/Male
n_female = sex_counts.get('F', 0)
n_male   = sex_counts.get('M', 0)
print(f"There are {n_female} females and {n_male} males in the study")

There are 35 females and 29 males in the study


## Age at first sampling, and under-18 vs adults

In [6]:
age_first = persons['Age']

median_age = age_first.median()
age_q1 = age_first.quantile(0.25)
age_q3 = age_first.quantile(0.75)

n_under18 = (age_first < 18).sum()
n_adult   = (age_first >= 18).sum()

age_group_counts = persons['Age groups'].value_counts()
paed_adult_counts = persons['Paediatric vs Adult'].value_counts()
n_under18

37

## Pancreatic insufficiency and CFRD / impaired glucose tolerance

In [7]:
# Pancreatic insufficiency
pi_counts = persons['Pancreatic insufficiency (PI)'].value_counts()
# e.g. if "Yes" marks PI:
n_pi = pi_counts.get('Yes', 0)

# CFRD / IGT (impaired glucose tolerance)
cfrd_counts = persons['Cystic Fibrosis related diabetes (CFRD)'].value_counts()

# Example if the column uses labels like "CFRD", "Impaired glucose tolerance", "Normal"
n_cfrd = cfrd_counts.get('Yes', 0)
n_igt  = cfrd_counts.get('Impaired', 0)

## Pseudomonas

In [8]:
meta['CS_Pseudo_num'] = meta['CS_Pseudomonas aeruginosa'].astype('Int64')
pseudo_any = meta.groupby('pwCF_ID')['CS_Pseudo_num'].max()

n_pseudo_positive = (pseudo_any == 1).sum()
n_pseudo_negative = (pseudo_any == 0).sum()
print(f"There are {n_pseudo_positive} Pseudomonas +ve samples and {n_pseudo_negative} Pseudomonas negative samples")

There are 28 Pseudomonas +ve samples and 36 Pseudomonas negative samples


In [9]:
meta['on_ivacaftor'] = meta['3 Ivacaftor (Kalydeco)'].astype('Int64')
iva_any = meta.groupby('pwCF_ID')['on_ivacaftor'].max()
n_on_iva = (iva_any == 1).sum()
n_off_iva = (iva_any == 0).sum()
print(f"There are {n_on_iva} people on ivacaftor and {n_off_iva} people not on ivacaftor")

There are 1 people on ivacaftor and 63 people not on ivacaftor


In [10]:
meta[meta['3 Ivacaftor (Kalydeco)']==1].index[0]

'1862551_20180123_S'

## IP vs OP

In [11]:
ip_counts = meta['IP vs OP'].value_counts()
n_inpatient_samples  = ip_counts.get('IP', 0)
n_outpatient_samples = ip_counts.get('OP', 0)

## Longitudinal coverage

In [12]:
sample_counts_per_person = meta['pwCF_ID'].value_counts()
n_longitudinal = (sample_counts_per_person >= 2).sum()


## Summary table

In [13]:
def combine_genes(a, b):
    return " / ".join(sorted([str(a), str(b)]))

persons['CF_genotype'] = persons.apply(
    lambda row: combine_genes(row['CF gene 1'], row['CF gene 2']),
    axis=1
)
genotype_counts = persons['CF_genotype'].value_counts()
# Choose top genotypes (you can pick how many to show)
top_genotypes = genotype_counts.head(5)

# Make a formatted string
genotype_summary_str = "; ".join(
    [f"{gt}: {count} ({count/len(persons):.0%})"
     for gt, count in top_genotypes.items()]
)

# Optionally add "other"
other = len(persons) - top_genotypes.sum()
if other > 0:
    genotype_summary_str += f"; Others: {other}"



In [14]:
summary_rows = [
    ("Number of participants", n_people),
    ("Number of samples", n_samples),
    ("Study period",
     f"{start_date.date()} – {end_date.date()} ({duration_days} days)"),
    ("Sex, female / male",
     f"{n_female} ({n_female/n_people:.0%}) / {n_male} ({n_male/n_people:.0%})"),
    ("Median age at first sampling (IQR)",
     f"{median_age:.1f} years ({age_q1:.1f}–{age_q3:.1f})"),
    ("Participants <18 years / ≥18 years",
     f"{n_under18} ({n_under18/n_people:.0%}) / {n_adult} ({n_adult/n_people:.0%})"),
    ("Pancreatic insufficiency",
     f"{n_pi} ({n_pi/n_people:.0%})"),
    ("CFRD",
     f"{n_cfrd} ({n_cfrd/n_people:.0%})"),
    ("Impaired glucose tolerance",
     f"{n_igt} ({n_igt/n_people:.0%})"),
    ("Pseudomonas infection",
     f"{n_pseudo_positive} ({n_pseudo_positive/n_people:.0%})"),
    ("Inpatient samples",
     f"{n_inpatient_samples}"),
    ("Participants with ≥2 samples",
     f"{n_longitudinal}"),
    ("CFTR genotype (top five)",
    genotype_summary_str)
]

summary_table = pd.DataFrame(summary_rows,
                             columns=["Characteristic", "Value"])
summary_table.to_excel("summary_table.xlsx", sheet_name="Metadata Summary", index=False)

In [15]:
summary_table

,Characteristic,Value
0,Number of participants,64
1,Number of samples,127
2,Study period,2017-09-18 – 2019-02-28 (528 days)
3,"Sex, female / male",35 (55%) / 29 (45%)
4,Median age at first sampling (IQR),16.5 years (11.0–26.5)
5,Participants <18 years / ≥18 years,37 (58%) / 27 (42%)
6,Pancreatic insufficiency,59 (92%)
7,CFRD,9 (14%)
8,Impaired glucose tolerance,7 (11%)
9,Pseudomonas infection,28 (44%)


In [16]:
genotype_summary_str

'F508 / F508: 41 (64%); ? / F508: 7 (11%); F508 / G551D: 4 (6%); A455 / F508: 1 (2%); F508 / p.Arg851: 1 (2%); Others: 10'